# FIT3182 - Assignment 1 - Questions Workbook

This assignment consists of **four questions** that are, taken together, worth 10% of the final marks. **Please treat this Assignment as a take-home test. This is an individual assignment and you should complete the questions on your own**. You should not attempt to post questions on EdForum seeking solutions to the answers. If you require clarification on the Assignment questions, you can post a private post on EdForum or seek consultation from the tutors.

1. The first question is related to **Parallel Search Algorithms (2 Mark)**.
2. The second question is related to **Parallel Join Algorithms (4 Marks)**.
3. The third question is related to **Parallel Sort Algorithms (2 Marks)**.
4. The fourth question is related to **Parallel GroupBy Algorithms (2 Marks)**.

**Instructions:**
- You will be using Python 3. Answer all questions inside this Jupyter Notebook. Please use the provided Docker to load the Jupyter Notebook.
- Read the instructions, skeleton code, and comments carefully.
- There are **code blocks that you need to complete** yourself as a part of the assignment. 
- You are also required to **answer the questions below**. 
- **Comment each line of code properly such that the tutor can easily understand what you are trying to do in the code. Marks may be deducted for insufficient or unclear comments.**
- Once completed, please rename the Jupyter Notebook to include your Student ID at the beginning of the filename (e.g., 12345678_FIT3182_Assignment_1.ipynb). Submit this Jupyter Notebook into the Assignment 1 submission link in Moodle. Please refer to the Assignment 1 submission link (or page) in Moodle for information about the submission due date.

**Please include your details as follows:**
- Student ID: 
- Last Name: 
- First (or Preferred) Name:
- Monash Student Email: 

In [100]:
from pprint import pprint

## Dataset
For this test, we will use the following tables R and S to write solutions to three parallel algorithms.

In [101]:
# R consists of 15 pairs, each comprising two attributes (nominal and numeric)
R = [('Adele',8),('Bob',22),('Clement',16),('Dave',23),('Ed',11),
     ('Fung',25),('Goel',3),('Harry',17),('Irene',14),('Joanna',2),
     ('Kelly',6),('Lim',20),('Meng',1),('Noor',5),('Omar',19)]

# S consists of 8 pairs, each comprising two attributes (nominal and numeric)
S = [('Arts',8),('Business',15),('CompSc',2),('Dance',12),('Engineering',7),
     ('Finance',21),('Geology',10),('Health',11),('IT',18)]

## 1. Parallel Searching Algorithm (2 marks)
**This section consist of 4 sub-questions.**

In this task, you will build a **parallel search algorithm for range selection (continuous)** for a given query.

 **Implement a parallel search algorithm** that uses local linear search (i.e., `linear_search()`) and is able to work with the hash partitioning method (i.e., `h_partition()`).
 **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


### 1.1 Local Search

In [102]:
# Linear search function
def linear_search(data, key):
    """
    Perform linear search on data for the given key.

    Arguments:
    data -- an input dataset (list OR numpy array)
    key -- an query attribute (number)

    Return:
    result --  list of matched records such as [('Adele', 8)]
                or the position of searched record
                whichever is more appropriate for your code
    """
    matches = []
    
    ### START CODE HERE ###
    for i in data:
        if i[1] == key:
            matches.append(i)
    
    ### END CODE HERE ###
    
    return matches

In [103]:
results = linear_search(R, 20)
print(results)

[('Lim', 20)]


### 1.2 Hash Partition

In [104]:
# Define a simple hash function.
def s_hash(x, n):
    """
    Define a simple hash function for demonstration

    Arguments:
    x -- an input record attribute (int)
    n -- the number of processors (int)

    Return:
    result -- the hash value of x
    """
    return x % n

In [105]:
# Hash data partitioning function. 
# We will use the s_hash function defined above
def h_partition(data, n):
    """
    Perform hash data partitioning on data

    Arguments:
    data -- an input dataset (list)
    n -- the number of processors (int)

    Return:
    result -- the partitioned subsets of D
    """
    partitions = {p:[] for p in range(n)}
    
    ### START CODE HERE ###
    for x in data:
        h=s_hash(x[1],n)
        partitions[h].append(x)
    
    ### END CODE HERE ###
    
    return partitions

In [106]:
partitions = h_partition(R, 5)
pprint(partitions)

{0: [('Fung', 25), ('Lim', 20), ('Noor', 5)],
 1: [('Clement', 16), ('Ed', 11), ('Kelly', 6), ('Meng', 1)],
 2: [('Bob', 22), ('Harry', 17), ('Joanna', 2)],
 3: [('Adele', 8), ('Dave', 23), ('Goel', 3)],
 4: [('Irene', 14), ('Omar', 19)]}


### 1.3 Parallel Search

In [107]:
from multiprocessing import Pool

def collect_result(results, result):
    # Callback for when linear_search returns a result.
    # The results list is modified only by the main process, not the pool workers.
    results.append(result)

# Parallel searching algorithm for range selection
def parallel_search_range(data, query_range, n_processor):
    """
    Perform parallel search for range selection on data for the given key.

    Arguments:
    data -- the input dataset (list)
    query_range -- a tuple with inclusive range end-points (e.g. [30, 50] means the interval 30-50)
    n_processor -- the number of parallel processors (int)
    
    Return:
    results -- the matched record information
    """
    results=[]
    pool = Pool(processes=n_processor)
    new_data = h_partition(data,n_processor)
        
    ### START CODE HERE ###    
    callbacks=[]
    for i in range (query_range[0],query_range[1]+1):
        query_hash = s_hash(i, n_processor)
#         print(i,new_data[query_hash])
        callbacks.append(pool.apply_async(linear_search,(list(new_data[query_hash]),i)))
    for item in callbacks:
#         print(item)
        for i in item.get():
            collect_result(results,i)
    ### END CODE HERE ###
    
    return results

In [108]:
n_processor = 3
results = parallel_search_range(R, [5, 15], n_processor)
pprint(results)

[('Noor', 5), ('Kelly', 6), ('Adele', 8), ('Ed', 11), ('Irene', 14)]


### 1.4 Parallel Search Variants

**Briefly answer the following.**

1. How would the parallel search you implemented need to be changed if we switched to round-robin partition for the data? Provide a small snippet of code with the modified search loop.


**Your Answers**:
    By using Round Robin partition we will not be able to find the specific partition to search so we need to loop through 3 of the partition. But for the hash partition we can first hashed the query key to get the specific partition and then we just need to search the value in the partition.
    
    #Snippet of code#
    new_data = RoundRobin_partition(data,n_processor)
          
    callbacks=[]
    for i in range (query_range[0],query_range[1]+1):
        for each_partition in new_data:
            callbacks.append(pool.apply_async(linear_search,(each_partition,i)))


## 2. Parallel Join Algorithm (4 marks)
**This section consist of 5 sub-questions.**

In this task, you will implement a **disjoint-partitioning based parallel join algorithm**. This algorithm consist of two stages.
1. Data disjoint-partitioning across processors.
1. Local join in each processor.

As a data partitioning method, use the range partitioning algorithm  (i.e. `range_partition( )`). Assume that we have **3 parallel processors**, processor 1 will get records with join attribute value between 1 and 9, processor 2 between 10 and 19, and processor 3 between 20 and 29. Note that both tables R and S need to be partitioned using the same ranges described earlier.

As a joining technique, use the hash based join algorithm (i.e., `HB_join( )`).  **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**

### 2.1 Range Partition

In [109]:
# Range data partitionining function (Need to modify as instructed above)
def range_partition(data, range_indices):
    """
    Perform range data partitioning on data based on the join attribute

    Arguments:
    data -- an input dataset (list)
    range_indices -- the range end-points (e.g., [5, 10, 15] means
                        4 intervals: x < 5, 5 <= x < 10, 10 <= x < 15, 15 <= x)

    Return:
    result -- the partitioned subsets of D
    """
    result = []
    ### START CODE HERE ###  
    new_data = list(data)
    new_data = sorted(new_data,key=lambda x: x[1])
    n_bin = len(range_indices)
    
    for i in range(n_bin): 
        s = [x for x in new_data if x[1] < range_indices[i]] 
        result.append(s) 
        last_element = s[len(s)-1]
        last = new_data.index(last_element)
        new_data = new_data[int(last)+1:] 

        
    result.append(new_data) 
    
    
    ### END CODE HERE ###
    
    return result

In [110]:
pprint(range_partition(R, [10, 20]))

[[('Meng', 1),
  ('Joanna', 2),
  ('Goel', 3),
  ('Noor', 5),
  ('Kelly', 6),
  ('Adele', 8)],
 [('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19)],
 [('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]]


### 2.2 Hash Join

In [111]:
def H(r):
    """
    We define a hash function 'H' that is used in the hashing process works 
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute. 
    
    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """
    
    # Convert the value of the join attribute into the digits
    digits = [int(d) for d in str(r[1])]
    
    # Calulate the sum of elemenets in the digits
    return sum(digits)

In [112]:
def HB_join(T1, T2):
    """
    Perform the hash-based join algorithm.
    The join attribute is the numeric attribute (in second position) in the input tables T1 & T2.

    Arguments:
    T1 & T2 -- Tables to be joined

    Return:
    result -- the joined table
    """
    result = []
    dic = {} # We will use a dictionary
    
    # For each record in table T2
    for s in T2:
        # Hash the record based on join attribute value using hash function H into hash table
        s_key = H(s)
        if s_key in dic:
            dic[s_key].add(s) # If there is an entry
        else:
            dic[s_key] = {s}
    print(dic)
    # For each record in table T1 (probing)
    for r in T1:
        # Hash the record based on join attribute value using H
        r_key = H(r)
        # If an index entry is found Then
        if r_key in dic:
            # Compare each record on this index entry with the record of table T1
            for item in dic[r_key]:
                if item[1] == r[1]:
                    # Put the rsult
                    result.append( (r[0], r[1], item[0]) )
                    
    return result

In [113]:
# print the partitioned result
HB_join(R,S)

{8: {('Arts', 8)}, 6: {('Business', 15)}, 2: {('Health', 11), ('CompSc', 2)}, 3: {('Dance', 12), ('Finance', 21)}, 7: {('Engineering', 7)}, 1: {('Geology', 10)}, 9: {('IT', 18)}}


[('Adele', 8, 'Arts'), ('Ed', 11, 'Health'), ('Joanna', 2, 'CompSc')]

### 2.3 Parallel Join

In [114]:
# Include this package for parallel processing
import multiprocessing as mp

def consolidate_result(results, result):
    # This is called whenever HB_Join(T1, T2) returns a result.
    # The results list is modified only by the main process, not the pool workers.
    results.append(result)

def DPBP_join(T1, T2, n_processor):
    """
    Perform a disjoint partitioning-based parallel join algorithm.
    The join attribute is the numeric attribute in the input tables T1 & T2

    Arguments:
    T1 & T2 -- Tables to be joined
    n_processor -- the number of parallel processors

    Return:
    result -- the joined table
    """
    
    ### START CODE HERE ###
    NT1= range_partition(T1,[10,20])
    NT2 = range_partition(T2,[10,20])
    pool = mp.Pool(processes = n_processor)
    
    callbacks=[]   
    results=[]
    # Apply local join for each processor
    for i in range (len(NT1)):
        callbacks.append(pool.apply_async(HB_join, [NT1[i], NT2[i]]))
    for callback in callbacks:
        for i in callback.get():
            results.append(i)
    ### END CODE HERE ###
    return results

In [115]:
n_processor = 3
DPBP_join(R, S, n_processor)

{2: {('CompSc', 2)}, 7: {('Engineering', 7)}, 8: {('Arts', 8)}}{1: {('Geology', 10)}, 2: {('Health', 11)}, 3: {('Dance', 12)}, 6: {('Business', 15)}, 9: {('IT', 18)}}{3: {('Finance', 21)}}




[('Joanna', 2, 'CompSc'), ('Adele', 8, 'Arts'), ('Ed', 11, 'Health')]

### 2.4 Duplication Outer Join Algorithm (DOJA)

In this task, you will implement a **Duplication Outer Join Algorithm (DOJA)**. This algorithm consist of four main steps.
1. (Replication): Duplicate the small table 
2. (Local Inner Join): Perform inner join in each processor
3. (Distribution): Reshuffle the inner join result based on R.x
4. (Local Outer Join): Perform outer join between the initial table R and the inner join result

Assume **n_processor=3** and the dataset is initially partition using the **hash** method. For joining technique, please use the provided **outer join** function for the inner join and outer join operation. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


In [116]:
# R consists of 15 pairs, each comprising two attributes (nominal and numeric)
R = [('Adele',8),('Bob',22),('Clement',16),('Dave',23),('Ed',11),
     ('Fung',25),('Goel',3),('Harry',17),('Irene',14),('Joanna',2),
     ('Kelly',6),('Lim',20),('Meng',1),('Noor',5),('Omar',19)]

# S consists of 8 pairs, each comprising two attributes (nominal and numeric)
S = [('Arts',8),('Business',15),('CompSc',2),('Dance',12),('Engineering',7),
     ('Finance',21),('Geology',10),('Health',11),('IT',18)]

In [117]:
import numpy as np

def H(r):
    """
    We define a hash function 'H' that is used in the hashing process works 
    by summing the first and second digits of the hashed attribute, which
    in this case is the join attribute. 
    
    Arguments:
    r -- a record where hashing will be applied on its join attribute

    Return:
    result -- the hash index of the record r
    """
    
    # Convert the value of the join attribute into the digits
    digits = [int(d) for d in str(r[1])]
    
    # Calulate the sum of elemenets in the digits
    return sum(digits)

def outer_join(L, R, join="left"):
    """outer join using Hash-based join algorithm"""
    
    if join == "right":
        L, R = R, L
    
    # inner join
    if join == "inner":
        h_dic = {}
        for r in R:
            h_r = H(r)
            if h_r in h_dic.keys():
                h_dic[h_r].add(r)
            else:
                h_dic[h_r] = {r}

        result = []
        for l in L:
            h_l = H(l)
            if h_l in h_dic.keys():
                for item in h_dic[h_l]:
                    if item[1] == l[1]:
                        result.append([l[0], item[1], item[0]])
        return result
    
    elif join in ["left", "right"]:
        h_dic = {}
        for r in R:
#             print(r)
            h_r = H(r)
            if h_r in h_dic.keys():
                h_dic[h_r].add(r)
            else:
                h_dic[h_r] = {r}
#         print(h_dic)
                
        result = []
        for l in L:
            isFound = False # to check whether there is a match found.
            h_l = H(l)

            if h_l in h_dic.keys():
                for item in h_dic[h_l]:
                    if item[1] == l[1]:
                        result.append([l[0], item[1], item[2]])
                        isFound = True
                        break
                        
            if not isFound:
                result.append([l[0], l[1],str(np.nan)])
        return result
    
    
    else:
        raise AttributeError('join should be in {left, right, inner}.') 
        
    

In [118]:
def hash_distribution(T, n):
    """distribute data using hash partitioning"""
    
    # Define a simple hash function for demonstration
    def s_hash(x, n):
        h = x%n 
        return h
    
    result = {p:[] for p in range(n)}
    for t in T:
        h_key = s_hash(t[1], n)
        result[h_key].append(tuple(t))
            
    return result

In [119]:
import multiprocessing as mp

def DOJA(L, R, n):
    """left outer join using DOJA"""
    
    ### Start CODE
    
    pool = mp.Pool(processes =n)
    l_dis = hash_distribution(L,n)
    r_dis = hash_distribution(R,n)
    first_Call=[]
#     print(L)
#     print(r_dis)
    first_Call=[]
    if len(L)<=len(R):
        Dup_lst=L
    else:
        Dup_lst = R
    for i in range(n):
        first_Call.append(pool.apply_async(outer_join,[Dup_lst,r_dis[i],"inner"]))
    
    output=[]
    for x in first_Call:
        for i in x.get():
            
            output.append(i)
    o_dis = hash_distribution(output,n)
#     print(o_dis)
    callbacks=[]
    for r in range(n):
        callbacks.append(pool.apply_async(outer_join,[l_dis[r],o_dis[r],"left"]))
        
    
    output=[]
    for x in callbacks:
            for i in x.get():
                output.append(i)
#     ### End CODE
    
    return output

In [120]:
DOJA(R,S,3)

[['Goel', 3, 'nan'],
 ['Kelly', 6, 'nan'],
 ['Bob', 22, 'nan'],
 ['Clement', 16, 'nan'],
 ['Fung', 25, 'nan'],
 ['Meng', 1, 'nan'],
 ['Omar', 19, 'nan'],
 ['Adele', 8, 'Arts'],
 ['Dave', 23, 'nan'],
 ['Ed', 11, 'Health'],
 ['Harry', 17, 'nan'],
 ['Irene', 14, 'nan'],
 ['Joanna', 2, 'CompSc'],
 ['Lim', 20, 'nan'],
 ['Noor', 5, 'nan']]

### 2.5 Parallel Join Variants

Briefly answer the following question.

1. For each partitioning algorithm besides range-partitioning, state and justify whether we can use it with the code for disjoint-partitioning based parallel join above.


**Your answer**:
For hash partition we are able to do so since the code above is using it and we are ensure the position of each data located in which partition so that when we parallel join will not missed any of the data.

But for round-robin partition it will not able to ensure the position of the data in each partition so there might be chance where the results will missed some data that could be joined.

Other than this two the random-unequal data partition also faced the same problem as round-robin partition sinced it cannot ensure each position of the data and there might be chanced it missed out an data that could be joined.


## 3. Parallel Sorting Algorithm (2 marks)

In this task, you will implement **parallel binary-merge sort** method. It has two phases same as the parallel merge-all sort that you learnt in the labs.
1. Local sort
2. Final merge.

The first phase is similar to the parallel merge-all sort. The second phase, the merging phase, is pipelined instead of concentrating on one processor. In this phase, we take the results from two processors and then merge the two in one processor, called binary merging. The result of the merge between two processors is passed on to the next level until one processor (the host) is left.

 **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**
Assume that we use the round robin partitioning method  (i.e. `rr_partition()`). 

### 3.1 Round-robin Partition

In [121]:
# Round-robin data partitionining function
def rr_partition(data, n):
    """
    Perform data partitioning on data

    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors

    Return:
    result -- the paritioned subsets of D
    """
    result = [[] for i in range (n)]
    
    ### START CODE HERE ### 
    for index, element in enumerate(data): 

        index_bin = (int) (index % n)
        result[index_bin].append(element)
    
    ### END CODE HERE ###
    
    return result

In [122]:
# Test the round-robin partitioning function
result = rr_partition(R, 3)
pprint(result)

[[('Adele', 8), ('Dave', 23), ('Goel', 3), ('Joanna', 2), ('Meng', 1)],
 [('Bob', 22), ('Ed', 11), ('Harry', 17), ('Kelly', 6), ('Noor', 5)],
 [('Clement', 16), ('Fung', 25), ('Irene', 14), ('Lim', 20), ('Omar', 19)]]


### 3.2 Serial Sort

In [123]:
def qsort(arr): 
    """ 
    Quicksort a list
    
    Arguments:
    arr -- the input list to be sorted

    Return:
    result -- the sorted arr
    """
    if len(arr) <= 1:
        return arr
    else:
        # take the first element as the pivot
        pivot = arr[0]
        left_arr = [x for x in arr[1:] if x[1] < pivot[1]]
        right_arr = [x for x in arr[1:] if x[1] >= pivot[1]]
        # uncomment this to see what to print 
        # print("Left:" + str(left_arr)+" Pivot : "+ str(pivot)+" Right: " + str(right_arr))
        sorted_arr = qsort(left_arr) + [pivot] + qsort(right_arr)
        
        return sorted_arr

In [124]:
qsort(R)

[('Meng', 1),
 ('Joanna', 2),
 ('Goel', 3),
 ('Noor', 5),
 ('Kelly', 6),
 ('Adele', 8),
 ('Ed', 11),
 ('Irene', 14),
 ('Clement', 16),
 ('Harry', 17),
 ('Omar', 19),
 ('Lim', 20),
 ('Bob', 22),
 ('Dave', 23),
 ('Fung', 25)]

In [125]:
# Let's first look at 'k-way merging algorithm' that will be used 
# to merge sub-record sets in our external sorting algorithm.
import sys

# Find the smallest record
def find_min(records):    
    """ 
    Find the smallest record
    
    Arguments:
    records -- the input record set

    Return:
    result -- the smallest record's index
    """
    m = records[0]
    index = 0
    for i in range(len(records)):
        if(records[i][1] < m[1]):  
            index = i
            m = records[i]
    return index

def k_way_merge(record_sets):
    """ 
    K-way merging algorithm
    
    Arguments:
    record_sets -- the set of multiple sorted sub-record sets

    Return:
    result -- the sorted and merged record set
    """
    # indexes will keep the indexes of sorted records in the given buffers
    indexes = []
    for _ in record_sets:
        indexes.append(0) # initialisation with 0

    # final result will be stored in this variable
    result = []
    while(True):
        merged_result = [] # the merging unit (i.e. # of the given buffers)
        
        # This loop gets the current position of every buffer
        for i in range(len(record_sets)):
            if(indexes[i] >= len(record_sets[i])):
                merged_result.append((None, sys.maxsize))
            else:
                merged_result.append(record_sets[i][indexes[i]])  
        
        # find the smallest record 
        smallest = find_min(merged_result)
        
        # if we only have sys.maxsize on the tuple, we reached the end of every record set
        if(merged_result[smallest][1] == sys.maxsize):
            break

        # This record is the next on the merged list
        result.append(record_sets[smallest][indexes[smallest]])
        indexes[smallest] +=1
   
    return result

In [126]:
# Test k-way merging method
buffers = rr_partition(R, 3)
print(buffers)
result = k_way_merge([qsort(b) for b in buffers])
pprint(result)

[[('Adele', 8), ('Dave', 23), ('Goel', 3), ('Joanna', 2), ('Meng', 1)], [('Bob', 22), ('Ed', 11), ('Harry', 17), ('Kelly', 6), ('Noor', 5)], [('Clement', 16), ('Fung', 25), ('Irene', 14), ('Lim', 20), ('Omar', 19)]]
[('Meng', 1),
 ('Joanna', 2),
 ('Goel', 3),
 ('Noor', 5),
 ('Kelly', 6),
 ('Adele', 8),
 ('Ed', 11),
 ('Irene', 14),
 ('Clement', 16),
 ('Harry', 17),
 ('Omar', 19),
 ('Lim', 20),
 ('Bob', 22),
 ('Dave', 23),
 ('Fung', 25)]


In [127]:
def serial_sorting(dataset, buffer_size):
    """
    Perform a serial external sorting method based on sort-merge
    The buffer size determines the size of each sub-record set

    Arguments:
    dataset -- the entire record set to be sorted
    buffer_size -- the buffer size determining the size of each sub-record set

    Return:
    result -- the sorted record set
    """
    
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return
    
    result = []
    ### START CODE HERE ### 
    sorted_set = []
    start_pos =0
    N= len(dataset)
    
    while True :
        if start_pos +buffer_size <N:
            subset = dataset[start_pos: start_pos +buffer_size]
            sorted_set.append(qsort(subset))
            start_pos +=buffer_size
            
        else:
            subset = dataset[start_pos:]
            sorted_set.append(qsort(subset))
            break
#     print(sorted_set)
    dataset = sorted_set
    merge_buffer_size = buffer_size -1
    while True:
        start_pos =0
        merged_set = []
        N=len(dataset)
        
        while True:
            if start_pos +buffer_size <N:
                subset = dataset[start_pos: start_pos +merge_buffer_size]
                sorted_set.append(k_way_merge(subset))
                start_pos +=merge_buffer_size
            
            else:
                subset = dataset[start_pos:]
                merged_set.append(k_way_merge(subset))
                break
    
        dataset =  merged_set
        if len(dataset)==1:
            break
    result=dataset
    ### END CODE HERE ###
    
    return result

In [128]:
print(R)

[('Adele', 8), ('Bob', 22), ('Clement', 16), ('Dave', 23), ('Ed', 11), ('Fung', 25), ('Goel', 3), ('Harry', 17), ('Irene', 14), ('Joanna', 2), ('Kelly', 6), ('Lim', 20), ('Meng', 1), ('Noor', 5), ('Omar', 19)]


In [129]:
result = serial_sorting(R, 4)
print("final sorting result:" + str(result))

final sorting result:[[('Meng', 1), ('Joanna', 2), ('Goel', 3), ('Noor', 5), ('Kelly', 6), ('Adele', 8), ('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19), ('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]]


### 3.3 Parallel Binary-merge Sort

In [144]:
# Include this package for parallel processing
import multiprocessing as mp

def parallel_binary_merge_sorting(dataset, n_processor, buffer_size):
    """
    Perform a parallel binary-merge sorting method

    Arguments:
    dataset -- entire record set to be sorted
    n_processor -- number of parallel processors
    buffer_size -- buffer size determining the size of each sub-record set

    Return:
    result -- the merged record set
    """
    
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return
    
    result = []

    ### START CODE HERE ### 
    subsets = rr_partition(dataset,n_processor)
    
    pool = mp.Pool(processes= n_processor)
    
    sorted_set = []
    callbacks = []
    for s in subsets:
        callbacks.append(pool.apply_async(serial_sorting,[s,buffer_size]))
    
    for c in callbacks:
        sorted_set += c.get()
        
    
    while len(sorted_set)!=1:
        
        temp_lst=[]
        callbacks_m=[]
#         print(len(sorted_set))
        for i in range (len(sorted_set)//2):
             callbacks_m.append(pool.apply_async(k_way_merge,[[sorted_set[i*2],sorted_set[i*2+1]]]))
        temp_lst=[r.get() for r in callbacks_m]
        if (i*2+1)!= len(sorted_set)-1:
            temp_lst.append(sorted_set[-1])
#         print(temp_lst)
        sorted_set=temp_lst
    result = sorted_set
            
    ### END CODE HERE ###
    
    return result

In [145]:
result = parallel_binary_merge_sorting(R, 10, 20)
print("final result:" + str(result))

final result:[[('Meng', 1), ('Joanna', 2), ('Goel', 3), ('Noor', 5), ('Kelly', 6), ('Adele', 8), ('Ed', 11), ('Irene', 14), ('Clement', 16), ('Harry', 17), ('Omar', 19), ('Lim', 20), ('Bob', 22), ('Dave', 23), ('Fung', 25)]]


### 3.4 Parallel Binary-merge Behavior

Briefly answer the following.

1. Does parallel binary-merge utilize all available processors? State and justify with respect to each phase: sort, merge.


**Your answer**:
For the sort phase we utilize all availble processors since we partition by using the number of processors but for the merging state we using binary merge so for the first merging step we only use 5 of the processors since 10/2 = 5 and each time we merge we will decrease the usage of processors by halved so we aren't using all available processors for merging state. 

## 4 Parallel GroupBy (2 marks)

### 4.1 Parallel GroupBy with Merge-All

In this task, you will implement **Parallel GroupBy with Merge-All** method. It invloves the following two steps:
1. Local aggregate in each processor
2. Global aggregation

Assume **n_processor=4** and the dataset has already been pre-partitioned. For local aggregation, please use the provided **local_groupby** function. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**


In [52]:
D1 = [('A', 1), ('B', 2), ('C', 3), ('A', 10), ('B', 20), ('C', 30)]
D2 = [('A', 2), ('B', 3), ('C', 4), ('A', 20), ('B', 30), ('C', 40)]
D3 = [('A', 3), ('B', 4), ('C', 5), ('A', 30), ('B', 40), ('C', 50)]
D4 = [('A', 4), ('B', 5), ('C', 6), ('A', 40), ('B', 50), ('C', 60)]

In [53]:
# The first step in the merge-all groupby method
def local_groupby(dataset):
    """
    Perform a local groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record set according to the group_by attribute index
    """

    dict = {}
    for index, record in enumerate(dataset):
        key = record[0]
        val = record[1]
        if key not in dict:
            dict[key] = 0
        dict[key] += val
    return dict

In [54]:
result = local_groupby (D1)
print(result)

{'A': 11, 'B': 22, 'C': 33}


In [55]:
import multiprocessing as mp

def parallel_merge_all_groupby(dataset):
    """
    Perform a parallel merge_all groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """
    
    result = {}

    ### START CODE HERE ### 
    n_processor = len(dataset)
    
    pool = mp.Pool(processes = n_processor)
    
    local_result = []
    callbacks=[]
    for s in dataset:
        callbacks.append(pool.apply_async(local_groupby, [s]))
    local_result=[r.get() for r in callbacks]
    pool.close()
#     print(local_result)
    
    
    for r in local_result:
        for key in r.keys():
            if key not in result:
                result[key]=0
            result[key] +=r[key]
       
    
    ### END CODE HERE ###
    
    return result

In [56]:
E = [D1, D2, D3, D4]
result = parallel_merge_all_groupby (E)
print(result)

{'A': 110, 'B': 154, 'C': 198}


### 4.2 Redistribution Method

In this task, you will implement **Parallel GroupBy with Redistribution Method** method. It invloves the following two steps:
1. (Partitioning phase): Redistribute raw records to all processor
2. (Aggregation phase): Each processor performs a local aggregation


Assume **n_processor=4** and the dataset should be partitioned into 4 groups using **range partition** Where first group with A & B, second with C & D, third with E & F and fourth with G & H. For local aggregation, please use the provided **local_groupby** function. **Complete the code block between `### START CODE HERE ###` and `### END CODE HERE ###`.**

In [57]:
D = [('A', 1), ('B', 2), ('C', 3), ('D', 10), ('E', 20), ('F', 30), ('G', 20), ('H', 30), 
      ('A', 2), ('B', 3), ('C', 4), ('D', 20), ('E', 30), ('F', 40), ('G', 30), ('H', 40),
      ('A', 3), ('B', 4), ('C', 5), ('D', 30), ('E', 40), ('F', 50), ('G', 40), ('H', 50),
      ('A', 4), ('B', 5), ('C', 6), ('D', 40), ('E', 50), ('F', 60), ('G', 50), ('H', 60)]

#### Create range partition function based on GroupBy attribute

In [76]:
# Range data partitionining function (Need to modify as instructed above)
def range_partition(data, range_indices):
    """
    Perform range data partitioning on data based on the join attribute

    Arguments:
    data -- an input dataset (list of tuple)
    range_indices -- the range end-points (e.g., ['C', 'E', 'G'] means
                        4 intervals: x < 'C', 'C' <= x < 'E', 'E' <= x < 'G', 'G' <= x)

    Return:
    result -- the partitioned subsets of D
    """
    result = [[] for i in range (len(range_indices)+1)]
    
    
    ### START CODE HERE ###  
    for i in D:
        flag=False
        for j in range (len(range_indices)):
            if i[0]<range_indices[j]:
                result[j].append(i)
                flag = True
    
                break
        if flag != True:
            result[j+1].append(i)
        
            
            
    
    ### END CODE HERE ###
    
    return result

In [88]:
import multiprocessing as mp

def parallel_redistributed_groupby(dataset, range_indices):
    """
    Perform a parallel redistributed groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """
    
    result = []
    
    ### START CODE HERE ### 
    distributed_set = range_partition(dataset,range_indices)
#     print(distributed_set)
    pool = mp.Pool(processes= len(range_indices)+1)
    local_result=[]
    callbacks=[]
    for i in distributed_set:
        callbacks.append(pool.apply_async(local_groupby,[i]))
    local_result=[r.get() for r in callbacks]
    pool.close()
#     print(local_result)
    merge_dic={}
    for i in local_result:
        merge_dic.update(i)
    result.append(merge_dic)

    ### END CODE HERE ###
    
    return result

In [89]:
result = parallel_redistributed_groupby (D, ['C', 'E', 'G'])
print(result)

[{'A': 10, 'B': 14, 'C': 18, 'D': 100, 'E': 140, 'F': 180, 'G': 140, 'H': 180}]
